# District Heating Data

In [9]:
import json
import os

import polars as pl

from src.energy_forecast.config import RAW_DATA_DIR, DATA_DIR

In [10]:
dh_data_folder = DATA_DIR / "district_heating_data"
dh_data_folder_merge = dh_data_folder / "data"
dh_data_folder_first_part = dh_data_folder / "2024_01_29 Projekt KI-FW Data Export (2022_08-2024_01_29)" / "data"
dh_data_folder_second_part = dh_data_folder / "2024_05_14 Projekt KI-FW Data Export (2024_02_08-2024_05_14)"
file_path_dump_1 = dh_data_folder_first_part / "dump.csv"
file_path_dump_2 = dh_data_folder_second_part / "dump.csv"

In [11]:
df = pl.scan_csv([file_path_dump_1, file_path_dump_2]).collect()
df

time,value,sensor_id,eco_u_id,data_provider_id
str,f64,str,str,str
"""2022-08-23T06:51:43.631Z""",0.0,"""d5209239-0c22-4a31-8bcc-0af1a0…","""c00c8cba-b6de-4c10-89c0-e92312…","""86237bdbtc1i"""
"""2022-08-23T07:52:09.604Z""",0.0,"""d5209239-0c22-4a31-8bcc-0af1a0…","""c00c8cba-b6de-4c10-89c0-e92312…","""86237bdbtc1i"""
"""2022-08-23T11:52:30.431Z""",0.0,"""d5209239-0c22-4a31-8bcc-0af1a0…","""c00c8cba-b6de-4c10-89c0-e92312…","""86237bdbtc1i"""
"""2022-08-23T13:52:12.392Z""",0.0,"""d5209239-0c22-4a31-8bcc-0af1a0…","""c00c8cba-b6de-4c10-89c0-e92312…","""86237bdbtc1i"""
"""2022-08-23T15:52:51.276Z""",3900.0,"""d5209239-0c22-4a31-8bcc-0af1a0…","""c00c8cba-b6de-4c10-89c0-e92312…","""86237bdbtc1i"""
…,…,…,…,…
"""2024-05-14T03:18:08.646Z""",25703.69,"""63d3721c-f850-4eb0-98ce-1c16b3…","""a9644794-439b-401c-b879-8c0225…","""607e3978xesq"""
"""2024-05-14T04:18:32.647Z""",25704.53,"""63d3721c-f850-4eb0-98ce-1c16b3…","""a9644794-439b-401c-b879-8c0225…","""607e3978xesq"""
"""2024-05-14T05:18:04.935Z""",25705.19,"""63d3721c-f850-4eb0-98ce-1c16b3…","""a9644794-439b-401c-b879-8c0225…","""607e3978xesq"""


Filter only the Data for data endpoints of type "FW Wärmemenge" and store one .csv file for each sensor

In [12]:
for file in os.listdir(dh_data_folder_first_part):
    filename = os.fsdecode(file)
    filename, file_extension = os.path.splitext(filename)
    print(filename)
    if file_extension == ".json":
        with open(dh_data_folder_first_part / file, "r", encoding="UTF-8") as f:
            eco_u_data = json.loads(f.read())
        for data_point in eco_u_data["datapoint_data"]:
            if data_point["title"] == "FW Wärmemenge":  # filter FW Wärmemenge
                sensor_id = data_point["id"]
                print(sensor_id)
        eco_u_id, data_provider_id = filename.split(".")
        df_sensor = df.filter(pl.col("eco_u_id") == eco_u_id,
                              pl.col("data_provider_id") == data_provider_id,
                              pl.col("sensor_id") == sensor_id
                              ).sort(pl.col("time"))
        print(len(df_sensor))
        store_csv_path = dh_data_folder_merge / f"{filename}.csv"
        df_sensor.write_csv(store_csv_path)
        print("Created " + str(store_csv_path))

8f7b3862-a50d-44eb-8ac9-de0cf48a6bd2.8ab8b58beel3
a04d39d6-4772-4cb0-8359-5c9b4f8ca9a5
8097
Created /home/marja/PycharmProjects/energy-forecast-wahl/data/district_heating_data/data/8f7b3862-a50d-44eb-8ac9-de0cf48a6bd2.8ab8b58beel3.csv
42d6efdc-d590-40b7-af9a-90121d04d3a9.0bcadf46b2ic
101016e7-4ef8-46fb-8a44-bdca27eeb39c
11977
Created /home/marja/PycharmProjects/energy-forecast-wahl/data/district_heating_data/data/42d6efdc-d590-40b7-af9a-90121d04d3a9.0bcadf46b2ic.csv
c00c8cba-b6de-4c10-89c0-e923127acc4e.71758690flyq
98c3ccff-f40a-4d28-b08d-5ae0f3189789
9265
Created /home/marja/PycharmProjects/energy-forecast-wahl/data/district_heating_data/data/c00c8cba-b6de-4c10-89c0-e923127acc4e.71758690flyq.csv
7bdbc8ee-00fb-4795-99cb-c1739fc128ad.5c852fb74usj
e8bbf035-5195-454e-a3d0-da9ae8f492c8
10120
Created /home/marja/PycharmProjects/energy-forecast-wahl/data/district_heating_data/data/7bdbc8ee-00fb-4795-99cb-c1739fc128ad.5c852fb74usj.csv
d00d6502-a08d-45df-99e3-7d8cd55200d1.b92753adeari
9f24aefc

Create one metadata json for all sensors included in the data now

In [13]:
eco_u_data_merge = dict()
for file in os.listdir(dh_data_folder_first_part):
    filename = os.fsdecode(file)
    filename, file_extension = os.path.splitext(filename)
    print(filename)
    if file_extension == ".json":
        with open(dh_data_folder_first_part / file, "r", encoding="UTF-8") as f:
            eco_u_data = json.loads(f.read())
        eco_u_data_point = {filename: eco_u_data["economic_unit"]}
        eco_u_data_merge.update(eco_u_data_point)
json_object = json.dumps(eco_u_data_merge)
with open(dh_data_folder / "eco_u_ids.json", "w") as outfile:
    outfile.write(json_object)

8f7b3862-a50d-44eb-8ac9-de0cf48a6bd2.8ab8b58beel3
42d6efdc-d590-40b7-af9a-90121d04d3a9.0bcadf46b2ic
c00c8cba-b6de-4c10-89c0-e923127acc4e.71758690flyq
7bdbc8ee-00fb-4795-99cb-c1739fc128ad.5c852fb74usj
d00d6502-a08d-45df-99e3-7d8cd55200d1.b92753adeari
8e9b1544-434e-44a7-8049-8f2e4b14a819.f1af853af3qs
c1469372-a3c4-4870-808c-8039cb927db8.7ce81adfg4dr
edcafda6-fe6f-4ca3-bb3d-f0c5fbac8f4c.338908912uns
4edd9f9b-22a0-4932-a815-60c9a222826d.22c3f026bbcr
bc098a2e-0cc7-4f01-b6ad-9d647ae9f627.cec5ec7cspn2
4f36b3bd-337e-4b93-9333-c53a28d0c417.a540555dmo6i
5e2fd59d-603a-488b-a525-513541039c4a.47e757c2y9pw
fb5cc271-ae15-4f24-b9d5-30782bbc2089.42fe7d9fz6oj
44201958-2d6b-4952-956c-22ea951a6442.779fa186y9wj
2a5d5a37-a843-4621-b20a-a0c1a5e42e49.61f9a1bcgzpi
10af300b-a270-4e41-928d-e4048b2fdf00.2e5dd0d6bct7
2a5d5a37-a843-4621-b20a-a0c1a5e42e49.dc62aac1q9uf
72013db2-014c-4c4f-8eee-ce179b77ba6f.9f5b73d65fya
7bdbc8ee-00fb-4795-99cb-c1739fc128ad.efd4921fwjck
fb5cc271-ae15-4f24-b9d5-30782bbc2089.91cabaaewpqb


Read in filtered data as new DataFrame

In [14]:
df = pl.scan_csv(dh_data_folder_merge / "*.csv").collect()
df

time,value,sensor_id,eco_u_id,data_provider_id
str,f64,str,str,str
"""2022-09-30T10:19:50.926Z""",1.806548e6,"""70a9a928-5277-44e9-b2dc-a4b06c…","""0c9ad311-b86f-4371-a695-512ca4…","""bd95e281dopi"""
"""2022-09-30T11:20:18.905Z""",1.806572e6,"""70a9a928-5277-44e9-b2dc-a4b06c…","""0c9ad311-b86f-4371-a695-512ca4…","""bd95e281dopi"""
"""2022-09-30T12:20:37.938Z""",1.8066e6,"""70a9a928-5277-44e9-b2dc-a4b06c…","""0c9ad311-b86f-4371-a695-512ca4…","""bd95e281dopi"""
"""2022-09-30T13:20:52.883Z""",1.806629e6,"""70a9a928-5277-44e9-b2dc-a4b06c…","""0c9ad311-b86f-4371-a695-512ca4…","""bd95e281dopi"""
"""2022-09-30T15:20:34.869Z""",1.806684e6,"""70a9a928-5277-44e9-b2dc-a4b06c…","""0c9ad311-b86f-4371-a695-512ca4…","""bd95e281dopi"""
…,…,…,…,…
"""2024-05-14T03:23:31.668Z""",322163.0,"""88c26917-f737-4a42-9c23-1cb3c4…","""fb684f25-a63d-4d3e-9277-6d759b…","""75febb50vqdf"""
"""2024-05-14T04:23:56.631Z""",322168.0,"""88c26917-f737-4a42-9c23-1cb3c4…","""fb684f25-a63d-4d3e-9277-6d759b…","""75febb50vqdf"""
"""2024-05-14T05:23:35.631Z""",322174.0,"""88c26917-f737-4a42-9c23-1cb3c4…","""fb684f25-a63d-4d3e-9277-6d759b…","""75febb50vqdf"""


In [15]:
df.group_by(["sensor_id", "eco_u_id", "data_provider_id"]).agg()

sensor_id,eco_u_id,data_provider_id
str,str,str
"""9b6aaad4-9da0-4bd5-8c7b-23deb2…","""e7ad9b75-bc6c-4891-a8fd-45e393…","""199f74d54sol"""
"""b8057e10-277b-4202-ba39-13b427…","""c00c8cba-b6de-4c10-89c0-e92312…","""8924d797x0zt"""
"""43abe4b0-a3d5-4e46-8d05-f164d5…","""fb5cc271-ae15-4f24-b9d5-30782b…","""c31d84a91xcl"""
"""675cdec1-ba1f-474b-b6b7-afcf66…","""b6b63b91-da14-449d-b213-e6ef5c…","""2d49cd92zn2e"""
"""68c7438c-2d9a-4c44-9ab7-cf3534…","""4edd9f9b-22a0-4932-a815-60c9a2…","""2680dfd89lmu"""
…,…,…
"""9833859d-3004-4701-b042-122d93…","""fb5cc271-ae15-4f24-b9d5-30782b…","""c3efc5b6l7n3"""
"""eaf6aa66-a5ee-43d4-ae40-9c98d5…","""573a7d1e-de3f-49e1-828b-07d463…","""568f5531a8wz"""
"""a8926502-d9c9-4271-9a52-5b64e7…","""72013db2-014c-4c4f-8eee-ce179b…","""9f5b73d65fya"""


Get the data provider id for each eco_u_id where the value is the largest, meaning it is the main counter for the building, and filter the DataFrame

In [16]:
eco_u_id_main_c = df.group_by(pl.col(["eco_u_id", "data_provider_id"])).agg(pl.len(), pl.col("time").min().alias(
    "min_time"), pl.col("time").max().alias("max_time"), pl.col("value").max().alias("max_value")).group_by(
    "eco_u_id").agg(pl.col("data_provider_id").filter(pl.col("max_value") == pl.col("max_value").max())).with_columns(
    pl.col("data_provider_id").map_elements(lambda v: v[0], return_dtype=pl.String))
df = df.filter(
    pl.struct("eco_u_id", "data_provider_id").is_in(eco_u_id_main_c)  # filter only main counters
)
df

time,value,sensor_id,eco_u_id,data_provider_id
str,f64,str,str,str
"""2022-09-30T10:19:50.926Z""",1.806548e6,"""70a9a928-5277-44e9-b2dc-a4b06c…","""0c9ad311-b86f-4371-a695-512ca4…","""bd95e281dopi"""
"""2022-09-30T11:20:18.905Z""",1.806572e6,"""70a9a928-5277-44e9-b2dc-a4b06c…","""0c9ad311-b86f-4371-a695-512ca4…","""bd95e281dopi"""
"""2022-09-30T12:20:37.938Z""",1.8066e6,"""70a9a928-5277-44e9-b2dc-a4b06c…","""0c9ad311-b86f-4371-a695-512ca4…","""bd95e281dopi"""
"""2022-09-30T13:20:52.883Z""",1.806629e6,"""70a9a928-5277-44e9-b2dc-a4b06c…","""0c9ad311-b86f-4371-a695-512ca4…","""bd95e281dopi"""
"""2022-09-30T15:20:34.869Z""",1.806684e6,"""70a9a928-5277-44e9-b2dc-a4b06c…","""0c9ad311-b86f-4371-a695-512ca4…","""bd95e281dopi"""
…,…,…,…,…
"""2024-05-14T03:23:31.668Z""",322163.0,"""88c26917-f737-4a42-9c23-1cb3c4…","""fb684f25-a63d-4d3e-9277-6d759b…","""75febb50vqdf"""
"""2024-05-14T04:23:56.631Z""",322168.0,"""88c26917-f737-4a42-9c23-1cb3c4…","""fb684f25-a63d-4d3e-9277-6d759b…","""75febb50vqdf"""
"""2024-05-14T05:23:35.631Z""",322174.0,"""88c26917-f737-4a42-9c23-1cb3c4…","""fb684f25-a63d-4d3e-9277-6d759b…","""75febb50vqdf"""


Create hourly data

In [19]:
df = df.with_columns(pl.col("time").str.to_datetime())

SchemaError: invalid series dtype: expected `String`, got `datetime[μs, UTC]` for series with name `time`

In [21]:
df = df.sort(["eco_u_id", "data_provider_id", "time"])

In [22]:
df_hourly = df.group_by_dynamic(index_column="time", every="1h",
                                group_by=["sensor_id", "eco_u_id", "data_provider_id"]).agg(pl.col("value").max())
df_hourly

sensor_id,eco_u_id,data_provider_id,time,value
str,str,str,"datetime[μs, UTC]",f64
"""70a9a928-5277-44e9-b2dc-a4b06c…","""0c9ad311-b86f-4371-a695-512ca4…","""bd95e281dopi""",2022-09-30 10:00:00 UTC,1.806548e6
"""70a9a928-5277-44e9-b2dc-a4b06c…","""0c9ad311-b86f-4371-a695-512ca4…","""bd95e281dopi""",2022-09-30 11:00:00 UTC,1.806572e6
"""70a9a928-5277-44e9-b2dc-a4b06c…","""0c9ad311-b86f-4371-a695-512ca4…","""bd95e281dopi""",2022-09-30 12:00:00 UTC,1.8066e6
"""70a9a928-5277-44e9-b2dc-a4b06c…","""0c9ad311-b86f-4371-a695-512ca4…","""bd95e281dopi""",2022-09-30 13:00:00 UTC,1.806629e6
"""70a9a928-5277-44e9-b2dc-a4b06c…","""0c9ad311-b86f-4371-a695-512ca4…","""bd95e281dopi""",2022-09-30 15:00:00 UTC,1.806684e6
…,…,…,…,…
"""88c26917-f737-4a42-9c23-1cb3c4…","""fb684f25-a63d-4d3e-9277-6d759b…","""75febb50vqdf""",2024-05-14 03:00:00 UTC,322163.0
"""88c26917-f737-4a42-9c23-1cb3c4…","""fb684f25-a63d-4d3e-9277-6d759b…","""75febb50vqdf""",2024-05-14 04:00:00 UTC,322168.0
"""88c26917-f737-4a42-9c23-1cb3c4…","""fb684f25-a63d-4d3e-9277-6d759b…","""75febb50vqdf""",2024-05-14 05:00:00 UTC,322174.0


In [23]:
df_hourly.write_csv(RAW_DATA_DIR / "district_heating_hourly.csv")

Create daily data

In [24]:
df_daily = df.group_by_dynamic(index_column="time", every="1d",
                               group_by=["sensor_id", "eco_u_id", "data_provider_id"]).agg(pl.col("value").max())
df_daily

sensor_id,eco_u_id,data_provider_id,time,value
str,str,str,"datetime[μs, UTC]",f64
"""70a9a928-5277-44e9-b2dc-a4b06c…","""0c9ad311-b86f-4371-a695-512ca4…","""bd95e281dopi""",2022-09-30 00:00:00 UTC,1.806912e6
"""70a9a928-5277-44e9-b2dc-a4b06c…","""0c9ad311-b86f-4371-a695-512ca4…","""bd95e281dopi""",2022-10-01 00:00:00 UTC,1.807615e6
"""70a9a928-5277-44e9-b2dc-a4b06c…","""0c9ad311-b86f-4371-a695-512ca4…","""bd95e281dopi""",2022-10-02 00:00:00 UTC,1.807949e6
"""70a9a928-5277-44e9-b2dc-a4b06c…","""0c9ad311-b86f-4371-a695-512ca4…","""bd95e281dopi""",2022-10-03 00:00:00 UTC,1.80884e6
"""70a9a928-5277-44e9-b2dc-a4b06c…","""0c9ad311-b86f-4371-a695-512ca4…","""bd95e281dopi""",2022-10-04 00:00:00 UTC,1.809501e6
…,…,…,…,…
"""88c26917-f737-4a42-9c23-1cb3c4…","""fb684f25-a63d-4d3e-9277-6d759b…","""75febb50vqdf""",2024-05-10 00:00:00 UTC,322009.0
"""88c26917-f737-4a42-9c23-1cb3c4…","""fb684f25-a63d-4d3e-9277-6d759b…","""75febb50vqdf""",2024-05-11 00:00:00 UTC,322060.0
"""88c26917-f737-4a42-9c23-1cb3c4…","""fb684f25-a63d-4d3e-9277-6d759b…","""75febb50vqdf""",2024-05-12 00:00:00 UTC,322105.0


In [25]:
df_daily.write_csv(RAW_DATA_DIR / "district_heating_daily.csv")

Find all addresses of sensors

In [27]:
eco_u_data_file = DATA_DIR / "district_heating_data" / "eco_u_ids.json"
with open(eco_u_data_file, "r", encoding="UTF-8") as f:
    meta = json.loads(f.read())
meta

{'8f7b3862-a50d-44eb-8ac9-de0cf48a6bd2.8ab8b58beel3': {'foreign_identifier': 'Kielort 20',
  'id': '8f7b3862-a50d-44eb-8ac9-de0cf48a6bd2',
  'address': {'street_address': 'Kielort 20',
   'address_locality': 'Norderstedt',
   'postal_code': 22850,
   'address_country': 'DE'},
  'year_of_completion': None,
  'heated_area': None},
 '42d6efdc-d590-40b7-af9a-90121d04d3a9.0bcadf46b2ic': {'foreign_identifier': 'Moorbekstraße 19',
  'id': '42d6efdc-d590-40b7-af9a-90121d04d3a9',
  'address': {'street_address': 'Moorbekstraße 19',
   'address_locality': 'Norderstedt',
   'postal_code': 22846,
   'address_country': 'DE'},
  'year_of_completion': None,
  'heated_area': None},
 'c00c8cba-b6de-4c10-89c0-e923127acc4e.71758690flyq': {'foreign_identifier': 'Moorbekstraße 29',
  'id': 'c00c8cba-b6de-4c10-89c0-e923127acc4e',
  'address': {'street_address': 'Moorbekstraße 29',
   'address_locality': 'Norderstedt',
   'postal_code': 22846,
   'address_country': 'DE'},
  'year_of_completion': None,
  'heat

In [28]:
meta_data = list()
for key, item in meta.items():
    eco_u_id, data_provider_id = key.split(".")
    address = item["address"]["street_address"]
    city = item["address"]["address_locality"]
    postal_code = item["address"]["postal_code"]
    country = item["address"]["address_country"]
    meta_data.append({"eco_u_id": eco_u_id, "data_provider_id": data_provider_id, "address": address, "city": city,
                      "postal_code": postal_code, "country": country})
df_meta = pl.DataFrame(meta_data)
df_meta

eco_u_id,data_provider_id,address,city,postal_code,country
str,str,str,str,str,str
"""8f7b3862-a50d-44eb-8ac9-de0cf4…","""8ab8b58beel3""","""Kielort 20""","""Norderstedt""","""22850""","""DE"""
"""42d6efdc-d590-40b7-af9a-90121d…","""0bcadf46b2ic""","""Moorbekstraße 19""","""Norderstedt""","""22846""","""DE"""
"""c00c8cba-b6de-4c10-89c0-e92312…","""71758690flyq""","""Moorbekstraße 29""","""Norderstedt""","""22846""","""DE"""
"""7bdbc8ee-00fb-4795-99cb-c1739f…","""5c852fb74usj""","""Moorbekstraße 31""","""Norderstedt""","""22846""","""DE"""
"""d00d6502-a08d-45df-99e3-7d8cd5…","""b92753adeari""","""Moorbekstraße 17""","""Norderstedt""","""22846""","""DE"""
…,…,…,…,…,…
"""82a01deb-7c9d-4e87-a79a-4693bd…","""16b697efmhyo""","""Waldstraße 81""","""Norderstedt""","""22846""","""DE"""
"""cae17ef4-cfad-4446-8b09-3cf946…","""a6eb7639f2t5""","""Hasenstieg 13""","""Norderstedt""","""22846""","""DE"""
"""a9644794-439b-401c-b879-8c0225…","""607e3978xesq""","""Kielort 25""","""Norderstedt""","""22850""","""DE"""


In [29]:
df_meta.write_csv(RAW_DATA_DIR / "district_heating_meta.csv")

In [31]:
df_meta.select(["address", "city", "postal_code", "country"]).sort(["postal_code", "address"]).unique(
    ["address"]).write_csv(RAW_DATA_DIR / "district_heating_addresses.csv")

Check multiple dataproviders for one address

In [32]:
sensor_id = "882f402f-daf8-4dfb-8af0-6fbd5d439e22"
df.filter(pl.col("eco_u_id") == sensor_id).group_by(pl.col("data_provider_id")).agg(pl.len(),
                                                                                    pl.col("time").min().alias(
                                                                                        "min_time"),
                                                                                    pl.col("time").max().alias(
                                                                                        "max_time"),
                                                                                    pl.col("value").max().alias(
                                                                                        "max_value"))

data_provider_id,len,min_time,max_time,max_value
str,u32,"datetime[μs, UTC]","datetime[μs, UTC]",f64
"""6fb87ad8f6oi""",23180,2022-08-23 07:42:42.655 UTC,2024-05-14 07:45:02.385 UTC,1.126987e6


In [33]:
df_daily.filter(pl.col("eco_u_id") == sensor_id).plot.line(x="time", y="value", color="data_provider_id")

alt.Chart(...)